We extract stats for the following corpora

# EHRs
**Cardiology**
* CarTeksten
* SCAD
* DataTools4Heart
* DIGIN
* PobTriage
* ARGUS
* CCN
* HMC

**IBD**
* MDL_hackathon $\rightarrow$ prefix with 'IBD_'

**Kidney**
* ODIN  $\rightarrow$ prefix with 'NEFRO_'

**RA**
* D2TRA hackathon  $\rightarrow$ prefix with 'RA_'
* STRATAFIT $\rightarrow$ prefix with 'RA_'

# Other

* NtvG  $\rightarrow$ prefix with 'GEN_'
* HenW $\rightarrow$ prefix with 'GEN_'
* NHG $\rightarrow$ prefix with 'GEN_'
* Pubmed-abstract translation  $\rightarrow$ prefix with 'GEN_'

Translations
* MIMIC III/IV 
* eICU
* Apollo corpus

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas
import re
import gc
import pandas as pd

In [ ]:
re_split = re.compile(r"\W")
splitter = lambda x: re_split.split(x)

In [ ]:
output_folder = r'L:\lab_research\RES-Folder-UPOD\CarTeksten\G_Output\2_Data'

## SCAD

4Million tokens

In [ ]:
folder = r'T:\lab_research\RES-Folder-UPOD\SCAD\E_ResearchData\2_ResearchData\20240312'
SCAD = pandas.read_sas(folder+"/carokverslag_20240312.sas7bdat")

In [ ]:
TEXT_GROUPED.shape

In [ ]:
SCAD['QRI_Stelling'] = SCAD['QRI_Stelling'].str.decode(encoding='latin1')
SCAD['QRI_ValueAnswerTxt'] = SCAD['QRI_ValueAnswerTxt'].str.decode(encoding='latin1')
SCAD = SCAD.loc[SCAD.QRI_ValueAnswerTxt.notna()]
SCAD['Age'] = SCAD['Age'].astype(str)
SCAD['gender'] = SCAD['gender'].str.decode(encoding='latin1')
SCAD['gender'] = SCAD['gender'].map({'male': 'man', 'female': 'vrouw'})
SCAD['gender'] = "Geslacht:"+ SCAD['gender']



In [ ]:
SCAD_TEXT = (SCAD.loc[SCAD['QRI_ValueAnswerTxt'].notna(), ['QRI_Stelling', 'QRI_ValueAnswerTxt', 'gender']]\
                .apply(lambda x: x[0]+":"+x[1]+", "+x[2], axis=1).to_frame().rename(columns={0:'text'}))

In [ ]:
NEW_SCAD = SCAD[['studyId_0818', 'index1_date']].join(SCAD_TEXT, how='inner')

In [ ]:
TEXT_GROUPED = NEW_SCAD.groupby(['studyId_0818', 'index1_date']).text.apply(lambda x: '\n'.join(x)).reset_index()

In [ ]:
TEXT_GROUPED['text_words'] = TEXT_GROUPED['text'].apply(splitter)
TEXT_GROUPED['text_len'] = TEXT_GROUPED['text_words'].apply(len)

In [ ]:
TEXT_GROUPED['text_len'].hist(bins=30)

In [ ]:
print(f"Number of words: {TEXT_GROUPED.text_len.sum()}")

In [ ]:
unique_words = set()
for l in TEXT_GROUPED['text_words'].values:
    for w in l:
        unique_words.add(w)

In [ ]:
print(f"Number of unique words: {len(unique_words)}")

In [ ]:
TEXT_GROUPED[['index1_date', 'text']].to_parquet(output_folder+'\SCAD.parquet')

In [ ]:
del SCAD, SCAD_TEXT, TEXT_GROUPED

In [ ]:
gc.collect()

## CarTeksten

400Million tokens

In [ ]:
folder = r'L:\lab_research\RES-Folder-UPOD\CarTeksten\E_ResearchData\2_ResearchData\20240321'
CarTeksten = pandas.read_parquet(folder+"/vrglijsten_20240321.parquet")

In [ ]:
CarTeksten = CarTeksten[CarTeksten['QRI_ValueAnswerTxt'].apply(lambda x: x is not None)]

In [ ]:
CarTeksten = CarTeksten.assign(TEXT= CarTeksten[['QRE_description', 'QRE_category_display', 'QRE_name', 'QRI_Stelling','QRI_ValueAnswerTxt']].apply(lambda x: 
                                                                                                                f'Omschrijving:{x['QRE_description']}'+\
                                                                                                                f', Categorie:{x['QRE_category_display']}'+\
                                                                                                                f', Thema:{x['QRE_name']}'+\
                                                                                                                f', Stelling:{x['QRI_Stelling']}'+\
                                                                                                                f', Antwoord:{x['QRI_ValueAnswerTxt']}'
                                                                                                               , axis=1))

In [ ]:
CarTeksten['text_words'] = CarTeksten['TEXT'].apply(splitter)
CarTeksten['text_len'] = CarTeksten['text_words'].apply(len)

In [ ]:
print(f"Number of words: {CarTeksten.text_len.sum()}")

unique_words = set()
for l in CarTeksten['text_words'].values:
    for w in l:
        unique_words.add(w)
        
print(f"Number of unique words: {len(unique_words)}")

In [ ]:
CarTeksten['text_len'].hist(bins=30)


In [ ]:
CarTeksten[['QRR_created', 'TEXT']].to_parquet(output_folder+'/CARQ.parquet')

In [ ]:
del CarTeksten
gc.collect()

## DIGIN

145Million tokens

In [ ]:
DIGIN_ECHO = pd.read_sas(r'L:\lab_research\RES-Folder-UPOD\DIGIN\E_ResearchData\2_ResearchData\pat_echo_brief_20231018.sas7bdat')

In [ ]:
DIGIN_ECHO['DOCOMSCHR'] = DIGIN_ECHO['DOCOMSCHR'].str.decode('latin1')
DIGIN_ECHO['Brief_txt'] = DIGIN_ECHO['Brief_txt'].str.decode('latin1')
DIGIN_ECHO['Conclusions_ECHO'] = DIGIN_ECHO['Conclusions_ECHO'].str.decode('latin1')
DIGIN_ECHO['gender'] = DIGIN_ECHO['gender'].str.decode('latin1')

In [ ]:
DIGIN_ECHO = DIGIN_ECHO.assign(TEXT = DIGIN_ECHO[['DOCOMSCHR', 'Brief_txt', 'Conclusions_ECHO', 'gender', 'age']].apply(lambda x:
                                                        f"Document type:{x['DOCOMSCHR']}\n"+\
                                                        f"Brief:{x['Brief_txt']}\n"+\
                                                        f"Echo:{x['Conclusions_ECHO']}\n"+\
                                                        f"Geslacht:{x['gender']}, Leeftijd:{str(x['age'])}", axis=1))

In [ ]:
DIGIN_ECHO.head()

In [ ]:
DIGIN_ECHO['text_words'] = DIGIN_ECHO['TEXT'].apply(splitter)
DIGIN_ECHO['text_len'] = DIGIN_ECHO['text_words'].apply(len)

In [ ]:
print(f"Number of words: {DIGIN_ECHO.text_len.sum()}")
unique_words = set()
for l in DIGIN_ECHO['text_words'].values:
    for w in l:
        unique_words.add(w)
print(f"Number of unique words: {len(unique_words)}")

In [ ]:
DIGIN_ECHO[['create_dt', 'TEXT']].to_parquet(output_folder+"/DIGIN_echo.parquet")
del DIGIN_ECHO
gc.collect()

In [31]:
DIGIN_POLI = pd.read_sas(r'L:\lab_research\RES-Folder-UPOD\DIGIN\E_ResearchData\2_ResearchData\pat_poli_brief_20231018.sas7bdat')

In [32]:
# poli_date, age, gender
DIGIN_POLI = DIGIN_POLI.assign(DOCOMSCHR=DIGIN_POLI.DOCOMSCHR.str.decode('latin1'))
DIGIN_POLI = DIGIN_POLI.assign(SPECIALISM=DIGIN_POLI.SPECIALISM.str.decode('latin1'))
DIGIN_POLI = DIGIN_POLI.assign(Brief_txt=DIGIN_POLI.Brief_txt.str.decode('latin1'))
DIGIN_POLI = DIGIN_POLI.assign(OMSCHR_AGENDA=DIGIN_POLI.OMSCHR_AGENDA.str.decode('latin1'))
DIGIN_POLI = DIGIN_POLI.assign(NAAM_AGENDA=DIGIN_POLI.NAAM_AGENDA.str.decode('latin1'))
DIGIN_POLI = DIGIN_POLI.assign(gender=DIGIN_POLI.gender.str.decode('latin1'))
DIGIN_POLI = DIGIN_POLI.assign(OMSCHR_Afspraak=DIGIN_POLI.OMSCHR_Afspraak.str.decode('latin1'))

In [34]:
DIGIN_POLI = DIGIN_POLI.assign(TEXT=DIGIN_POLI[['DOCOMSCHR', 'SPECIALISM', 'NAAM_AGENDA', 'OMSCHR_Afspraak', 'gender', 'age', 'Brief_txt']].apply(lambda x:
                                            f"Document: {x['DOCOMSCHR']}\n"\
                                            f"Specialisme: {x['SPECIALISM']}\n"\
                                            f"Specialisme: {x['NAAM_AGENDA']}\n"\
                                            f"Afspraak: {x['OMSCHR_Afspraak']}\n"\
                                            f"Geslacht: {x['gender']}, Leeftijd: {str(x['age'])}\n"\
                                            f"Brief: {x['Brief_txt']}" ,axis=1))

In [36]:
DIGIN_POLI['text_words'] = DIGIN_POLI['TEXT'].apply(splitter)
DIGIN_POLI['text_len'] = DIGIN_POLI['text_words'].apply(len)

In [37]:
print(f"Number of words: {DIGIN_POLI.text_len.sum()}")
unique_words = set()
for l in DIGIN_POLI['text_words'].values:
    for w in l:
        unique_words.add(w)
print(f"Number of unique words: {len(unique_words)}")

Number of words: 6227734
Number of unique words: 51792


In [39]:
DIGIN_POLI[['poli_date', 'TEXT']].to_parquet(output_folder+"/DIGIN_POLI.parquet")
del DIGIN_POLI
gc.collect()

 ## DataTools4Heart

In [ ]:
folder = r'T:\lab_research\RES-Folder-UPOD\DataTools4Heart\E_ResearchData\2_ResearchData\20240709'
echo_conc = pd.read_sas(folder+"/echo_concl_20240709.sas7bdat")

folder = r'T:\lab_research\RES-Folder-UPOD\DataTools4Heart\E_ResearchData\2_ResearchData\20240715'
echo = pd.read_sas(folder+"/echo_20240715.sas7bdat")

#ecg = pd.read_sas(folder+'/ecg_20240610.sas7bdat')
#vital = pd.read_sas(folder+'/vitalsign_20240610.sas7bdat')

In [ ]:
echo.groupby(['studyId_0763', 'identifier_value']).size().hist(bins=20)

In [ ]:
echo_conc = echo_conc.assign(datum=echo_conc.effectiveDateTime.dt.date)

In [ ]:
echo_conc[['datum', 'conclusion']].to_parquet(output_folder+"/dt4h_echo.parquet")

## PobTriage

In [40]:
zdbrieven = r'T:\lab_research\RES-Folder-UPOD\PobTriage\E_ResearchData\2_ResearchData\20230130\ZD_brieven_20230130.parquet'
inkbrieven = r'T:\lab_research\RES-Folder-UPOD\PobTriage\E_ResearchData\2_ResearchData\Old\inkomendebrieven.sas7bdat'
verslagen = r'T:\lab_research\RES-Folder-UPOD\PobTriage\E_ResearchData\2_ResearchData\20230216\verslagen_20230216.sas7bdat'
ontslagbrieven = r'T:\lab_research\RES-Folder-UPOD\PobTriage\E_ResearchData\2_ResearchData\20230216\ontslagbrieven_20230216.sas7bdat'

zdbrieven_df = pd.read_parquet(zdbrieven)
inkbrieven_df = pd.read_sas(inkbrieven)
verslagen_df = pd.read_sas(verslagen)
ontslagbrieven_df = pd.read_sas(ontslagbrieven)

In [45]:
zdbrieven_df = zdbrieven_df.assign(zdbrieven_df[['InkomendeBriefTekst_DOC', 'age', 'gender']])

,studyId_0758,InkomendeBriefTekst_DOC,age,gender,jaar,brief_date
0,4.217040e+06,zorginstelling: 1499164 \t\nVerwijzing Datum...,69.0,F,2022.0,2022-01-24 16:14:00
1,4.797928e+06,KERN VERWIJSBRIEF\t\n \t\nreden van verwijzing...,67.0,F,2015.0,2015-04-03 15:22:00
2,8.424936e+06,KERN VERWIJSBRIEF\t\n \t\nreden van verwijzing...,70.0,F,2014.0,NaT
3,2.779259e+07,zorginstelling: 1493906 \t\nVerwijzing Datum...,63.0,M,2020.0,2020-08-26 11:41:00
4,6.045692e+07,KERNDEEL VERWIJSBRIEF\t\n05-02-2018 15:33\t\nG...,71.0,F,2018.0,2018-02-05 15:30:00
...,...,...,...,...,...,...
9642,7.083527e+10,zorginstelling: 5675855 \t\nVerwijzing Datum...,83.0,F,2019.0,2019-12-16 20:47:00
9643,7.083527e+10,zorginstelling: 5675855 \t\nVerwijzing Datum...,85.0,F,2021.0,2021-08-09 13:19:00
9644,7.083527e+10,zorginstelling: 5675855 \t\nVerwijzing Datum...,85.0,F,2021.0,2021-09-27 10:39:00
9645,7.083572e+10,zorginstelling: 5675864 \t\nVerwijzing Datum...,33.0,F,2019.0,2019-09-11 08:49:00


In [47]:
inkbrieven_df

,studyId_Ruijter_Argus,pateventid,reporttxt,ONDERZDAT,DOC_InkomendeBriefID,DOC_InkomendeBriefTekstID,DOC_InkomendeBriefTypeID,InkomendeBriefType_DOC,InkomendeBriefNaam_DOC,InkomendeBriefInvoerdatum_DOC,InkomendeBriefInvoertijd_DOC,InkomendeBriefAanmaakdatum_DOC,InkomendeBriefAanmaaktijd_DOC,InkomendeBriefSpecialisme_DOC,InkomendeBriefStatusCode_DOC,InkomendeBriefStatus_DOC,InkomendeBriefFileExt_DOC,InkomendeBriefTekst_DOC
0,2.078540e+05,b' 207854-02',b'_Klinische gegevens: Deelnemer MARC-2. SUBJE...,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,1.553076e+06,b' 1553076-01',b'_Klinische gegevens: 2 dagen oude jongen met...,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,1.156461e+07,b' 11564612-01',b'_Klinische gegevens: Uitsluiten coronairlijd...,2018-11-12,b'2001768778',b'2011532811',b'CS00000142',b'Zorgdomein verwijzingen',b'Zorgdomein ZD47530670',2018-10-17,43140.0,2018-10-17,42660.0,b'CAR',b'G',b'Geaccordeerd',b'.rtf',b'zorginstelling: 1467097 \t\nVerwijzing/aanv...
3,1.501103e+07,b' 15011027-01',b'_Klinische gegevens:\r\nVolgens SMART-ORACLE...,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,2.596922e+07,b' 25969220-02',b'_Klinische gegevens: Deelnemer MARC-2 studie...,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,5.176577e+09,b' 5176576783-01',b'_Klinische gegevens: uitsluiten coronairlijd...,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
832,5.180365e+09,b' 5180365495-02',b'_Klinische gegevens: transpositie grote vate...,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
833,5.193506e+09,b' 5193506157-03',b'_Klinische gegevens:. Bekend met tweemaal ab...,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
834,5.195021e+09,b' 5195021225-01',b'_Klinische gegevens: Agatston score? Coronai...,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
verslagen_df

,studyId_0758,indexdate,onderznr,indicatie,verr_jaar,transtext,tekst,aangemaakt,aanmtijd,gewijzigd,wijztijd,plattetext
0,NaN,2019-12-19,b'1007309119',b'Follow-up 6 maanden na OK. Aanwijzingen voor...,2020.0,b'_Klinische gegevens: Status na bilobectomie ...,NaN,2020-09-29,b'16:21',2020-09-29,b'16:21',NaN
1,NaN,2020-01-17,b'1007309119',b'Follow-up 6 maanden na OK. Aanwijzingen voor...,2020.0,b'_Klinische gegevens: Status na bilobectomie ...,NaN,2020-09-29,b'16:21',2020-09-29,b'16:21',NaN
2,NaN,2016-06-29,b'1006104642',b'afw in pulm? afw bij xiphoid?',2016.0,"b'_Klinische gegevens: status na 2x CABG, heef...",NaN,2016-09-02,b'08:10',2016-09-02,b'08:10',NaN
3,NaN,2021-02-05,b'1007655178',b'Afwijkingen?',2021.0,b'_Klinische gegevens: Trauma Indicatie: Afwij...,NaN,2021-10-14,b'20:30',2021-10-14,b'20:30',NaN
4,NaN,2021-02-05,b'1007658712',b'Traumatische afwijkingen? Bloeding?',2021.0,b'_Klinische gegevens: Bekend met recidiverend...,NaN,2021-10-19,b'18:26',2021-10-19,b'18:26',NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
12974,NaN,2020-12-31,b'1007474559',b'st. na CEA links. ACI rechts eerder stenose ...,2021.0,NaN,NaN,2021-06-20,b'14:52',2021-06-20,b'14:52',b' \rOnderzoek in kader...
12975,NaN,2020-12-31,b'1007532673',b'Vaten abdominaal/inguinaal doorgankelijk? He...,2021.0,"b'_Klinische gegevens: VG EVAR, CEA, CABG. Nu ...",NaN,2021-05-16,b'19:41',2021-05-16,b'19:41',NaN
12976,NaN,2021-01-04,b'1007532673',b'Vaten abdominaal/inguinaal doorgankelijk? He...,2021.0,b'_Addendum\r\nDe infiltratie van de perirenal...,NaN,2021-05-17,b'16:01',2021-05-17,b'16:01',NaN
12977,NaN,2018-01-30,b'1006573029',"b'infiltraat ,pneumonie'",2018.0,b'_Klinische gegevens: grieperig en koorts In...,NaN,2018-03-14,b'11:17',2018-03-14,b'11:17',NaN


In [49]:
ontslagbrieven_df

,studyId_0758,indexdate,docOmsch,plattetext,specialism,DOCTYPE,vervallen,create_dt,modify_dt
0,1.715535e+10,2022-03-17,b'Overigen',"b' \r\r ,\r\r\rHierbij willen wij u uitnodigen...",b'CAR',b'1000100009',0.0,2023-02-16 14:31:00,2023-02-16 14:32:00
1,6.846058e+10,2014-06-17,b'Poliklinische Brief',"b' \r\n\r\nGeachte collega,\r\n\r\nBovengenoem...",b'CAR',b'CS00000004',0.0,2015-06-02 16:17:00,2015-06-03 07:55:00
2,4.456282e+08,2014-07-01,b'Poliklinische Brief',"b' \r\nUMC Utrecht, Polikliniek Endocrinologie...",b'END',b'CS00000004',0.0,2015-06-02 16:43:00,2015-06-09 16:17:00
3,5.697212e+10,2014-12-05,b'SEH EDI Bericht',"b'UMC Utrecht, Afdeling Spoedeisende Hulp 088 ...",b'ANE',b'1000100061',0.0,2015-06-02 23:06:00,2015-06-02 23:06:00
4,6.126760e+08,2022-05-16,b'Poliklinische Brief',"b' \rUMC Utrecht, Polikliniek Vasculaire Genee...",b'VAS',b'CS00000004',0.0,2023-02-17 14:15:00,2023-02-20 09:04:00
...,...,...,...,...,...,...,...,...,...
37366,6.445946e+09,2021-02-08,b'Radiologie',NaN,b'RAD',b'CS00000001',0.0,2021-09-07 09:57:00,2021-09-07 09:57:00
37367,6.683572e+10,2018-11-26,b'Radiologie',NaN,b'RAD',b'CS00000001',0.0,2019-03-25 13:47:00,2019-03-25 13:47:00
37368,1.494086e+10,2015-04-16,b'Radiologie',NaN,b'RAD',b'CS00000001',0.0,2015-07-28 16:14:00,2015-07-28 16:14:00
37369,2.205471e+10,2014-04-30,b'Radiologie',NaN,b'RAD',b'CS00000001',0.0,2014-06-02 17:28:00,2014-06-02 17:28:00


## CCN

## HMC

## ARGUS